# Publishing position information from Gazebo for Multiple Robots

Sometimes localizing robots in Gazebo without using the navigation stack can be a headache. To deal with this problem, what we can do instead is launch a publisher node that gets the desired robot position from gazebo serivce and then pushes it into a rostopic. 

Gazebo has a service called "get_model_state" that can be used to obtain the position of various models in the simulation environment. The clearpath ridgeback package does not wholly support launching multiple robots currently. Hence setting up the navigation stack for muliple robots is a big headache. This shortcut helps us use gazebo as a localization system.

Full credits to the tutorial from Rickardo Tellez from Construct Sim.
[ ROS Q&A 007 - How to publish odometry from simulation position](https://www.youtube.com/watch?v=I_5leJK8vhQ)


## Launch the simulation in Gazebo with the correct namespaces

First step is to create a multi-robot launch file similar in structure(doesnt have to be a package) to [Multi Jackal Gazebo](https://github.com/NicksSimulationsROS/multi_jackal). Make sure to get the namespaces and the tf prefixes right. So that every topic being published is different for all the robots. 

## Check the service that generates position into Gazebo

Check the list of services by gazebo

In [ ]:
rosservice list

In [ ]:
rosservice call /gazebo/get_model_state

The above command publishes the position of models in Gazebo. Specify the robot model to get its position. In order to use it first get a list of model names.

In [ ]:
rosservice call /gazebo/get_world_properties

In [ ]:
rosservice call /gazebo/get_model_state '{model_name: ridge0}'

## Create a package for the publisher

In [ ]:
cd ~/ridgeback_ws/src
catkin_create_pkg gazebo_odom_publisher rospy
cd gazebo_odom_publisher
mkdir launch
gedit start_odom.launch

In [ ]:
<launch>
    <node name="odom_pub" pkg="gazebo_odom_publisher" type="gazebo_odom.py" output="screen"/>
</launch>

In [ ]:
cd gazebo_odom_publisher/src
gedit gazebo_odom.py

In [ ]:
#!/usr/bin/env python

import rospy
from nav_msgs.msg import Odometry
from std_msgs.msg import Header
from gazebo_msgs.srv import GetModelState, GetModelStateRequest

rospy.init_node('odom_pub')

# Base name should not contian / so 'gazeboOdom'
odom_pub = rospy.Publisher('gazeboOdom',Odometry,queue_size=10)

# If the service is not working the node will wait here.
rospy.wait_for_service('/gazebo/get_model_state')
get_model_srv = rospy.ServiceProxy('/gazebo/get_model_state',GetModelState)

# get msg from service and push into /gazeboOdom topic.

odom = Odometry()
header = Header()
header.frame_id = '/odom'

model = GetModelStateRequest()
# The following depends on your robot
model.model_name = 'mobile_base'

rate = rospy.Rate(10)

while not rospy.is_shutdown():
    result = get_model_srv(model)
    
    odom.pose.pose = result.pose
    odom.twist.twist = result.twist
    
    header.stamp = rospy.Time.now()
    odom.header = header
    odom_pub.publish(odom)
    
    rate.sleep()

In [ ]:
chmod +x gazebo_odom.py

In [ ]:
rossrv show gazebo_msgs/GetModelState
rosmsg show Odometry

In [ ]:
cd ../..
catkin_make

## Run the publisher node

In [ ]:
roslaunch gazebo_odom_publisher start_odom.launch

In [ ]:
rostopic list
rostopic echo /gazeboOdom

The above code helps publish the state of one model. In order to publish the positions of all the robots, 
the publisher created needs to accept the robot model as a param and publish data accordingly.

# Publisher for Multiple Robots

The following creates a new publisher called "gazebo_odom_multi.py" that accepts the name of the model whose state 
has to be published as a parameter. This parameter is loaded in the launch file. This way the same publisher code can be replicated to publish the states of different robots.

## Create the new publisher node and the launch file

In [ ]:
cd ~/ridgeback_ws/src/launch
gedit start_odom_six.launch

In [ ]:
<?xml version="1.0"?>
<launch>
 <node name="odom_pub_1" pkg="gazebo_odom_publisher" type="gazebo_odom_multi.py" output="screen">
    <param name="robotName" type="string" value="ridge0"/>
 </node>
 <node name="odom_pub_2" pkg="gazebo_odom_publisher" type="gazebo_odom_multi.py" output="screen">
    <param name="robotName" type="string" value="ridge1"/>
 </node>	
 <node name="odom_pub_3" pkg="gazebo_odom_publisher" type="gazebo_odom_multi.py" output="screen">
    <param name="robotName" type="string" value="ridge2"/>
 </node>
 <node name="odom_pub_4" pkg="gazebo_odom_publisher" type="gazebo_odom_multi.py" output="screen">
    <param name="robotName" type="string" value="ridge3"/>
 </node>
 <node name="odom_pub_5" pkg="gazebo_odom_publisher" type="gazebo_odom_multi.py" output="screen">
    <param name="robotName" type="string" value="ridge4"/>
 </node>
 <node name="odom_pub_6" pkg="gazebo_odom_publisher" type="gazebo_odom_multi.py" output="screen">
    <param name="robotName" type="string" value="ridge5"/>
 </node>
 
</launch>

As can be noted, the value of the param "robotName" corresponds to the "name" argument passed while launching multiple robots in gazebo.

In [ ]:
cd gazebo_odom_publisher/src
gedit gazebo_odom_multi.py

In [ ]:
#!/usr/bin/env python

import rospy
from nav_msgs.msg import Odometry
from std_msgs.msg import Header
from gazebo_msgs.srv import GetModelState, GetModelStateRequest

rospy.init_node('odom_pub')
robot_name = rospy.get_param('~robotName')

odom_pub = rospy.Publisher('/%s/gazeboOdom'% robot_name,Odometry,queue_size=10)

rospy.wait_for_service('/gazebo/get_model_state')
get_model_srv = rospy.ServiceProxy('/gazebo/get_model_state',GetModelState)

odom = Odometry()
header = Header()
header.frame_id = '/odom'

model = GetModelStateRequest()

model.model_name = robot_name

rate = rospy.Rate(10)

while not rospy.is_shutdown():
	result = get_model_srv(model)
	
	odom.pose.pose = result.pose
	odom.twist.twist = result.twist

	header.stamp = rospy.Time.now()
	odom.header = header
	odom_pub.publish(odom)

	rate.sleep()

In [ ]:
chmod +x gazebo_odom_multi.py

## Run the multi robot publisher node.

In [ ]:
roslaunch gazebo_odom_publisher start_odom_six.launch